In [1]:
import numpy as np
import pickle
import pandas as pd
import CMR_IA as cmr
import scipy as sp
import pingouin as pg
import sys

sys.path.append("../../Modeling/fitting")
from optimization_utils import make_boundary
from object_funcs import anal_perform_S1 as anal_perform

np.set_printoptions(suppress=True)

## Import Params from PSO

In [2]:
# read pso results
_, _, what_to_fit = make_boundary(sim_name="S1")
optim_params = np.loadtxt("simuS1_data/S1_240801_200-200.txt")

# get params
params = cmr.make_default_params()
for pname, pvalue in zip(what_to_fit, optim_params):
    params[pname] = pvalue
params.update(learn_while_retrieving=True, rec_time_limit=10000, use_new_context=True, nitems_in_accumulator=48 * 4)
params

{'beta_enc': 0.406351,
 'beta_rec': 0.037046,
 'beta_cue': 0.872296,
 'beta_rec_post': 0.978895,
 'beta_distract': 0.97831,
 'phi_s': 4.906676,
 'phi_d': 4.763418,
 's_cf': 0.01516,
 's_fc': 0.129968,
 'kappa': 0.297122,
 'eta': 0.000236,
 'omega': 9.709221,
 'alpha': 0.805698,
 'lamb': 0.022135,
 'c_thresh': 0.165399,
 'c_thresh_itm': 0.222,
 'c_thresh_assoc': 0.30722,
 'd_assoc': 1,
 'rec_time_limit': 10000,
 'dt': 10,
 'nitems_in_accumulator': 192,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'use_new_context': True,
 'psi_s': 0,
 'psi_c': 1,
 'c_s': 0,
 'thresh_sigma': 0,
 'ban_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'a': 2800,
 'b': 20,
 'gamma_fc': 0.201812,
 'gamma_cf': 0.801713}

## Define Simulation Function

In [3]:
def simu_success(tag, params):
    # which task
    path = "simuS1_data/simuS1_design.pkl"
    if tag == "Item-CR":
        test1_num = 80
        i = 1
    elif tag == "Pair-CR":
        test1_num = 80
        i = 2
    elif tag == "Asso-CR":
        test1_num = 40
        i = 3

    # load stimuli
    with open(path, "rb") as inp:
        df_study = pickle.load(inp)
        df_test = pickle.load(inp)
    df_study = df_study.query(f"group == {i}")
    df_test = df_test.query(f"group == {i}")

    # load semantic matrix
    s_mat = np.load("../wordpools/ltp_FR_similarity_matrix.npy")

    # run CMR
    df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params, df_study, df_test, s_mat)
    df_simu["test"] = df_test["test"]
    df_simu = df_simu.merge(df_test, on=["session", "list", "test", "test_itemno1", "test_itemno2"])

    # get f_in
    sessions = np.unique(df_simu.session)
    tmp_corr_fin = []
    tmp_omax_fin = []

    for sess in sessions:
        df_tmp = df_study.loc[df_study.session == sess]
        tmp1 = df_tmp.study_itemno1.to_numpy()
        tmp2 = df_tmp.study_itemno2.to_numpy()
        df_tmp2 = df_test.loc[df_test.session == sess]
        tmp3 = df_tmp2.test_itemno1[df_tmp2.test_itemno1 >= 0].to_numpy()
        tmp4 = df_tmp2.test_itemno2[df_tmp2.test_itemno2 >= 0].to_numpy()
        tmp = np.concatenate((tmp1, tmp2, tmp3, tmp4))
        tmp = np.unique(tmp)  # sort
        nlists = len(np.unique(df_simu.list))

        for lst in range(nlists):
            tmp_corr = df_simu.query(f"session == {sess} and list == {lst}")["correct_ans"][test1_num:]
            corrid = np.searchsorted(tmp, tmp_corr)
            corr_fin = [f_in_dif[sess][lst * int(test1_num / 2) + i][id] for i, id in enumerate(corrid)]
            omax_fin = [np.max(np.delete(f_in_dif[sess][lst * int(test1_num / 2) + i], id)) for i, id in enumerate(corrid)]

            tmp_corr_fin = tmp_corr_fin + [-1] * test1_num + corr_fin
            tmp_omax_fin = tmp_omax_fin + [-1] * test1_num + omax_fin

    df_simu["corr_fin"] = tmp_corr_fin
    df_simu["omax_fin"] = tmp_omax_fin

    return df_simu

## Results

### Item - CR (Group1)

In [4]:
# run the model
tag = "Item-CR"
df_simu_g1 = simu_success(tag, params)
df_simu_g1

100%|██████████| 100/100 [01:35<00:00,  1.04it/s]


CMR Time: 95.83108305931091


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,correct_ans,order,pair_idx,subject,group,test_item2,corr_fin,omax_fin
0,0,0,609,-1,0.0,177.676941,0.084130,1,FROST,1,0,14,0,1,NaN,-1.000000,-1.000000
1,0,0,101,-1,0.0,411.736847,0.126150,1,BATTERY,0,-1,-1,0,1,NaN,-1.000000,-1.000000
2,0,0,927,-1,0.0,1791.123291,0.199661,1,MOVIE,1,1,13,0,1,NaN,-1.000000,-1.000000
3,0,0,1303,-1,1.0,1089.297607,0.269204,1,SIDEWALK,1,0,31,0,1,NaN,-1.000000,-1.000000
4,0,0,945,-1,1.0,1018.410522,0.272569,1,NEST,1,0,2,0,1,NaN,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47995,99,3,509,-1,895.0,760.000000,0.538228,2,DUSTPAN,128,1,149,99,1,NaN,-0.415913,1.881162
47996,99,3,2,-1,909.0,880.000000,0.443602,2,ACID,970,1,163,99,1,NaN,-0.064730,1.039054
47997,99,3,1463,-1,730.0,890.000000,0.492720,2,TICK,1255,0,171,99,1,NaN,-0.057539,1.330427
47998,99,3,1615,-1,1065.0,1210.000000,0.364676,2,WINNER,667,1,157,99,1,NaN,-0.171165,0.538347


In [5]:
# analysis
subjects = np.unique(df_simu_g1.subject)
g1_stats = []
for subj in subjects:
    df_subj = df_simu_g1.query(f"subject == {subj}").copy()
    g1_stats.append(list(anal_perform(df_subj)))
g1_stats = np.array(g1_stats)

In [6]:
print(g1_stats)
print("mean:")
print(np.mean(g1_stats, axis=0))
print("se:")
print(sp.stats.sem(g1_stats, axis=0))

[[0.1125     0.56875    0.3        0.71017274]
 [0.10625    0.65       0.1875     0.58118858]
 [0.1        0.625      0.1625     0.27885921]
 [0.1125     0.69375    0.20625    0.53300733]
 [0.11875    0.60625    0.2        0.55551227]
 [0.11875    0.63125    0.225      0.51400731]
 [0.1125     0.51875    0.2375     0.65614916]
 [0.11875    0.6125     0.25       0.67722772]
 [0.1125     0.55625    0.16875    0.60567108]
 [0.11875    0.63125    0.20625    0.51400731]
 [0.11875    0.68125    0.33125    0.74381708]
 [0.125      0.6125     0.20625    0.44444444]
 [0.1125     0.6125     0.19375    0.51873199]
 [0.125      0.61875    0.23125    0.56      ]
 [0.08125    0.5875     0.09375    0.5607575 ]
 [0.11875    0.6375     0.23125    0.64449064]
 [0.125      0.6375     0.2375     0.66374851]
 [0.125      0.65625    0.30625    0.4957265 ]
 [0.1        0.66875    0.19375    0.52320245]
 [0.09375    0.6125     0.25625    0.26160338]
 [0.10625    0.60625    0.20625    0.50021377]
 [0.10625    

In [7]:
# hack hr and far
old_num = 40
new_num = 40
g1_stats_hack = g1_stats.copy()
hacked_hr = (g1_stats_hack[:, 1] * old_num + 0.5) / (old_num + 1)
hacked_far = (g1_stats_hack[:, 2] * new_num + 0.5) / (new_num + 1)
g1_stats_hack[:, 1] = hacked_hr
g1_stats_hack[:, 2] = hacked_far

# calculate d prime
g1_ds = sp.stats.norm.ppf(g1_stats_hack[:, 1]) - sp.stats.norm.ppf(g1_stats_hack[:, 2])
print(np.mean(g1_ds))
print(sp.stats.sem(g1_ds))

1.0454167493701687
0.015245030870168874


### Pair - CR (Group2)

In [8]:
# run the model
tag = "Pair-CR"
df_simu_g2 = simu_success(tag, params)
df_simu_g2

100%|██████████| 100/100 [01:16<00:00,  1.31it/s]


CMR Time: 76.53287506103516


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,correct_ans,order,pair_idx,subject,group,test_item2,corr_fin,omax_fin
0,0,0,192,1402,0.0,797.331055,0.284805,1,BULB,1,1,13,0,2,SUIT,-1.000000,-1.000000
1,0,0,810,1481,0.0,1623.264160,0.249259,1,LEMON,0,-1,-1,0,2,TOOTH,-1.000000,-1.000000
2,0,0,384,686,1.0,51.012386,0.422265,1,COTTAGE,1,1,8,0,2,HANDKERCHIEF,-1.000000,-1.000000
3,0,0,1317,1137,0.0,160.621506,0.079084,1,SKILLET,0,-1,-1,0,2,PUBLISHER,-1.000000,-1.000000
4,0,0,506,977,0.0,2005.499878,0.205314,1,DUNE,0,-1,-1,0,2,ONION,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,99,2,120,-1,649.0,2020.000000,0.414275,2,BELT,917,0,97,99,2,-1,-0.022449,0.204465
35996,99,2,328,-1,762.0,2970.000000,0.438397,2,CLOUD,869,0,113,99,2,-1,-0.422165,-0.057257
35997,99,2,456,-1,918.0,2030.000000,0.492994,2,DIAGRAM,1582,0,110,99,2,-1,0.216130,0.096920
35998,99,2,655,-1,1195.0,2000.000000,0.566386,2,GRASS,1520,0,114,99,2,-1,0.266867,0.252300


In [9]:
# analysis
subjects = np.unique(df_simu_g2.subject)
g2_stats = []
for subj in subjects:
    df_subj = df_simu_g2.query(f"subject == {subj}").copy()
    g2_stats.append(list(anal_perform(df_subj)))
g2_stats = np.array(g2_stats)

In [10]:
print(g2_stats)
print("mean:")
print(np.mean(g2_stats, axis=0))
print("se:")
print(sp.stats.sem(g2_stats, axis=0))

[[0.23333333 0.80833333 0.00833333 0.70807453]
 [0.425      0.80833333 0.00833333 0.70321199]
 [0.325      0.75       0.025      0.68061951]
 [0.325      0.775      0.         0.63258786]
 [0.33333333 0.79166667 0.00833333 0.60944206]
 [0.325      0.79166667 0.00833333 0.71532847]
 [0.24166667 0.8        0.01666667 0.5615142 ]
 [0.31666667 0.74166667 0.         0.77901694]
 [0.3        0.86666667 0.00833333 0.8923802 ]
 [0.33333333 0.75833333 0.         0.77568134]
 [0.33333333 0.79166667 0.00833333 0.60944206]
 [0.31666667 0.83333333 0.         0.61446503]
 [0.26666667 0.81666667 0.00833333 0.73988439]
 [0.28333333 0.70833333 0.         0.78118609]
 [0.33333333 0.775      0.         0.7523427 ]
 [0.35       0.75833333 0.00833333 0.87961859]
 [0.29166667 0.75833333 0.         0.72599532]
 [0.30833333 0.825      0.01666667 0.47146974]
 [0.35833333 0.74166667 0.00833333 0.8197696 ]
 [0.25       0.75833333 0.01666667 0.66312997]
 [0.39166667 0.7        0.         0.82633997]
 [0.36666667 

In [11]:
# hack hr and far
old_num = 40
new_num = 40
g2_stats_hack = g2_stats.copy()
hacked_hr = (g2_stats_hack[:, 1] * old_num + 0.5) / (old_num + 1)
hacked_far = (g2_stats_hack[:, 2] * new_num + 0.5) / (new_num + 1)
g2_stats_hack[:, 1] = hacked_hr
g2_stats_hack[:, 2] = hacked_far

# calculate d prime
g2_ds = sp.stats.norm.ppf(g2_stats_hack[:, 1]) - sp.stats.norm.ppf(g2_stats_hack[:, 2])
print(np.mean(g2_ds))
print(sp.stats.sem(g2_ds))

2.8132320144356293
0.018721023018474885


### Association - CR (Group3)

In [12]:
# run the model
tag = "Asso-CR"
df_simu_g3 = simu_success(tag, params)
df_simu_g3

100%|██████████| 100/100 [01:03<00:00,  1.58it/s]


CMR Time: 63.21623492240906


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,correct_ans,order,pair_idx,subject,group,test_item2,corr_fin,omax_fin
0,0,0,1213,926,0.0,1121.403931,0.267752,1,ROUGE,0,1,-1,0,3,MOUTH,-1.000000,-1.000000
1,0,0,830,717,0.0,693.153992,0.291806,1,LIST,1,1,34,0,3,HORIZON,-1.000000,-1.000000
2,0,0,577,1183,0.0,2637.572998,0.224988,1,FLEA,1,1,0,0,3,RELISH,-1.000000,-1.000000
3,0,0,588,439,1.0,256.345856,0.341542,1,FOLDER,1,0,25,0,3,DANDRUFF,-1.000000,-1.000000
4,0,0,687,1311,1.0,112.520996,0.382712,1,HANGER,1,1,22,0,3,SIRLOIN,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,99,4,1169,-1,-1.0,-1.000000,-1.000000,2,RAT,331,0,226,99,3,-1,-0.376510,0.223466
29996,99,4,1349,-1,-1.0,-1.000000,-1.000000,2,SPHINX,848,1,219,99,3,-1,0.200690,0.184725
29997,99,4,1307,-1,-1.0,-1.000000,-1.000000,2,SILK,72,0,194,99,3,-1,-0.529434,0.087847
29998,99,4,984,-1,-1.0,-1.000000,-1.000000,2,ORGAN,899,1,196,99,3,-1,0.168325,0.019125


In [13]:
# analysis
subjects = np.unique(df_simu_g3.subject)
g3_stats = []
for subj in subjects:
    df_subj = df_simu_g3.query(f"subject == {subj}").copy()
    g3_stats.append(list(anal_perform(df_subj)))
g3_stats = np.array(g3_stats)

In [14]:
print(g3_stats)
print("mean:")
print(np.mean(g3_stats, axis=0))
print("se:")
print(sp.stats.sem(g3_stats, axis=0))

[[0.39       0.82       0.22       0.6894198 ]
 [0.46       0.7        0.29       0.85106383]
 [0.4        0.73       0.26       0.83418963]
 [0.44       0.74       0.26       0.85523385]
 [0.45       0.72       0.14       0.93249112]
 [0.46       0.79       0.2        0.89487049]
 [0.4        0.73       0.17       0.90578061]
 [0.44       0.82       0.25       0.95430267]
 [0.55       0.76       0.13       0.85878963]
 [0.32       0.72       0.28       0.86591276]
 [0.41       0.84       0.18       0.80777538]
 [0.44       0.77       0.23       0.89956332]
 [0.43       0.62       0.15       0.89614243]
 [0.42       0.78       0.21       0.79222882]
 [0.42       0.78       0.2        0.79222882]
 [0.49       0.86       0.27       0.83789474]
 [0.45       0.73       0.18       0.92766653]
 [0.48       0.76       0.19       0.86414254]
 [0.52       0.84       0.28       0.68643533]
 [0.45       0.76       0.17       0.84235294]
 [0.46       0.78       0.34       0.90228488]
 [0.38       

In [15]:
# hack hr and far
old_num = 20
new_num = 20
g3_stats_hack = g3_stats.copy()
hacked_hr = (g3_stats_hack[:, 1] * old_num + 0.5) / (old_num + 1)
hacked_far = (g3_stats_hack[:, 2] * new_num + 0.5) / (new_num + 1)
g3_stats_hack[:, 1] = hacked_hr
g3_stats_hack[:, 2] = hacked_far

# calculate d prime
g3_ds = sp.stats.norm.ppf(g3_stats_hack[:, 1]) - sp.stats.norm.ppf(g3_stats_hack[:, 2])
print(np.mean(g3_ds))
print(sp.stats.sem(g3_ds))

1.4288822377532773
0.017558799793224653


### Aggregate Three Groups

In [16]:
stats = []
stats.append(list(np.mean(np.array(g1_stats), axis=0)))
stats.append(list(np.mean(np.array(g2_stats), axis=0)))
stats.append(list(np.mean(np.array(g3_stats), axis=0)))
stats = np.array(stats)
stats

array([[0.112875  , 0.6176875 , 0.2215625 , 0.58308398],
       [0.31816667, 0.77525   , 0.00825   , 0.71855385],
       [0.4128    , 0.7664    , 0.2196    , 0.80563758]])

In [17]:
ground_truth = np.array([[0.19, 0.67, 0.15, 0.57], [0.30, 0.80, 0.12, 0.71], [0.42, 0.72, 0.22, 0.81]])  # p_rc, hr, far, q
err = np.mean(np.power(stats - ground_truth, 2))
err

0.002475421406920775

### Performance Analysis

In [18]:
def array2df(group_stats, group_ds, group_num):
    group_stats = np.array(group_stats)
    df = pd.DataFrame(group_stats, columns=["PR", "HR", "FAR", "Q"])
    df["d"] = group_ds
    df["subject"] = df.index
    df["group"] = group_num
    return df


df_group1 = array2df(g1_stats, g1_ds, 1)
df_group2 = array2df(g2_stats, g2_ds, 2)
df_group3 = array2df(g3_stats, g3_ds, 3)
df_groups = pd.concat([df_group1, df_group2, df_group3])
df_groups

,PR,HR,FAR,Q,d,subject,group
0,0.11250,0.56875,0.30000,0.710173,0.679349,0,1
1,0.10625,0.65000,0.18750,0.581189,1.234637,1,1
2,0.10000,0.62500,0.16250,0.278859,1.261887,2,1
3,0.11250,0.69375,0.20625,0.533007,1.287716,3,1
4,0.11875,0.60625,0.20000,0.555512,1.078594,4,1
...,...,...,...,...,...,...,...
95,0.46000,0.75000,0.13000,0.975427,1.684184,95,3
96,0.51000,0.76000,0.15000,0.825383,1.634438,96,3
97,0.46000,0.72000,0.18000,0.936170,1.410748,97,3
98,0.43000,0.78000,0.18000,0.801482,1.586667,98,3


In [19]:
pg.anova(data=df_groups, dv="PR", between="group", detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,group,4.701838,2,2.350919,1173.048181,1.048462e-141,0.887632
1,Within,0.595221,297,0.002004,NaN,NaN,NaN


In [20]:
pg.pairwise_tukey(data=df_groups, dv="PR", between="group")

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,1,2,0.112875,0.318167,-0.205292,0.006331,-32.426152,0.0,-5.544352
1,1,3,0.112875,0.412800,-0.299925,0.006331,-47.373640,0.0,-7.176394
2,2,3,0.318167,0.412800,-0.094633,0.006331,-14.947489,0.0,-1.745150


In [21]:
pg.anova(data=df_groups, dv="d", between="group", detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,group,172.954697,2,86.477348,2911.044672,7.609283e-196,0.951463
1,Within,8.822871,297,0.029707,NaN,NaN,NaN


In [22]:
pg.pairwise_tukey(data=df_groups, dv="d", between="group")

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,1,2,1.045417,2.813232,-1.767815,0.024375,-72.526235,0.0,-10.315949
1,1,3,1.045417,1.428882,-0.383465,0.024375,-15.732022,0.0,-2.323292
2,2,3,2.813232,1.428882,1.384350,0.024375,56.794214,0.0,7.598672


In [23]:
pg.anova(data=df_groups, dv="Q", between="group", detailed=True)

,Source,SS,DF,MS,F,p-unc,np2
0,group,2.515526,2,1.257763,87.019507,1.800471e-30,0.369479
1,Within,4.292780,297,0.014454,NaN,NaN,NaN


In [24]:
pg.pairwise_tukey(data=df_groups, dv="Q", between="group")

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges
0,1,2,0.583084,0.718554,-0.135470,0.017002,-7.967767,2.331468e-15,-1.085418
1,1,3,0.583084,0.805638,-0.222554,0.017002,-13.089666,0.000000e+00,-1.767295
2,2,3,0.718554,0.805638,-0.087084,0.017002,-5.121899,1.629310e-06,-0.786590


### Symmetry Analysis

In [25]:
def test_sym(df_simu, testnum):
    df = df_simu.query(f"test == {testnum} and order >= 0").copy()
    df["correct"] = df["s_resp"] == df["correct_ans"]
    df_order = df.groupby(["order", "subject"]).correct.mean().to_frame(name="p_correct").reset_index()
    return df_order

In [26]:
df_simu_all = pd.concat([df_simu_g1, df_simu_g2, df_simu_g3])
df_order_recog = test_sym(df_simu_all, 1)
print(df_order_recog.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_recog, dv="p_correct", subject="subject", within="order")

order
0    0.723667
1    0.720250
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,0.855625,99.0,two-sided,0.394271,0.158,0.116139


In [27]:
df_order_g1_recog = test_sym(df_simu_g1, 1)
print(df_order_g1_recog.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g1_recog, dv="p_correct", subject="subject", within="order")

order
0    0.615750
1    0.619625
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,-0.530906,99.0,two-sided,0.596673,0.127,-0.067023


In [28]:
df_order_g1_cr = test_sym(df_simu_g1, 2)
print(df_order_g1_cr.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g1_cr, dv="p_correct", subject="subject", within="order")

order
0    0.112875
1    0.112875
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,-9.511297e-17,99.0,two-sided,1.0,0.111,5.402573e-16


In [29]:
df_order_g2_recog = test_sym(df_simu_g2, 1)
print(df_order_g2_recog.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g2_recog, dv="p_correct", subject="subject", within="order")

order
0    0.7835
1    0.7670
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,2.463372,99.0,two-sided,0.015489,1.949,0.329586


In [30]:
df_order_g2_cr = test_sym(df_simu_g2, 2)
print(df_order_g2_cr.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g2_cr, dv="p_correct", subject="subject", within="order")

order
0    0.324833
1    0.311500
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,2.184281,99.0,two-sided,0.031302,1.07,0.225262


In [31]:
df_order_g3_recog = test_sym(df_simu_g3, 1)
print(df_order_g3_recog.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g3_recog, dv="p_correct", subject="subject", within="order")

order
0    0.7741
1    0.7727
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,0.247252,99.0,two-sided,0.805225,0.114,0.034802


In [32]:
df_order_g3_cr = test_sym(df_simu_g3, 2)
print(df_order_g3_cr.groupby("order").p_correct.mean())
pg.pairwise_tests(df_order_g3_cr, dv="p_correct", subject="subject", within="order")

order
0    0.415601
1    0.410106
Name: p_correct, dtype: float64


,Contrast,A,B,Paired,Parametric,T,dof,alternative,p-unc,BF10,hedges
0,order,0,1,True,True,0.523711,99.0,two-sided,0.60165,0.127,0.070273
